In [1]:
!pip install deepchem

In [2]:
!pip install mordred

In [3]:
!pip install xgboost

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# DATA_PATH = "/content/drive/MyDrive/project/data/"
DATA_PATH = "C:\python-code\project_bio\data/"

SEED = 42

In [5]:
train = pd.read_csv(f"{DATA_PATH}train.csv")
test = pd.read_csv(f"{DATA_PATH}test.csv")

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True
from mordred import Calculator, descriptors

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn import tree
from flaml import AutoML

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [8]:
from sklearn.metrics import mean_squared_error

def rmse(y_valid, pred):
    mse = mean_squared_error(y_valid, pred)
    return np.sqrt(mse)

In [9]:
from sklearn.metrics import make_scorer

rmse_score = make_scorer(rmse, greater_is_better=False)

### -> 이상치 제거 (총 18개)

In [11]:
# 1. 극단치만 판단해서 18개 제거
outliers = train[(train['MLM'] > 100.0) | (train['HLM'] > 100.0) | (train['AlogP'] < -3) | (train['Molecular_Weight'] > 800)
     | (train['Num_H_Acceptors'] > 14) | (train['Num_H_Donors'] > 9) | (train['Num_RotatableBonds'] > 20)
     | (train['LogD'] < -4) | (train['Molecular_PolarSurfaceArea'] > 250)]


In [12]:
outliers.index

Index([ 179,  662,  834,  983, 1092, 1172, 1239, 1584, 2159, 2258, 2367, 2410,
       2586, 2711, 2948, 3157, 3247, 3403],
      dtype='int64')

In [13]:
train = train.drop(index=outliers.index, axis=0)

In [14]:
train.shape # 3498 -> 3480

(3480, 11)

In [15]:
duplicates = train[train.duplicated(subset=['SMILES'], keep=False)]

duplicates.sort_values(by='SMILES')


,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
2276,TRAIN_2276,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,1.535,31.453,3.556,262.309,3,0,4,3.556,43.60
451,TRAIN_0451,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.310,24.670,3.556,262.309,3,0,4,3.556,43.60
2891,TRAIN_2891,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,55.950,69.950,2.172,337.372,4,2,3,2.169,82.00
543,TRAIN_0543,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,68.485,85.872,2.172,337.372,4,2,3,2.169,82.00
837,TRAIN_0837,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,63.522,62.488,2.293,367.428,5,2,3,2.307,139.85
366,TRAIN_0366,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,73.740,66.850,2.293,367.428,5,2,3,2.307,139.85
1085,TRAIN_1085,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,23.630,71.470,1.684,381.450,7,1,4,1.684,126.52
2848,TRAIN_2848,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,40.657,99.900,1.684,381.450,7,1,4,1.684,126.52
2096,TRAIN_2096,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,1.880,61.280,2.843,360.490,3,0,4,2.843,49.85
1666,TRAIN_1666,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,5.494,25.397,2.843,360.490,3,0,4,2.843,49.85


In [17]:
processed_duplicate = duplicates.groupby('SMILES')[['MLM', 'HLM']].mean().reset_index()
processed_duplicate

,SMILES,MLM,HLM
0,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615
1,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110
2,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690
3,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850
4,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385
5,CC(C)NC(=O)c1c(Cl)nn(C)c1NC(=O)c1cc(Br)nn1-c1n...,62.1085,68.1015
6,CC1CC(=O)N(c2ccc(-c3cccc(C#N)c3)cc2)N=C1c1ccc(...,43.1700,31.1300
7,CCCCC/N=c1\n(C)c(=O)nc2sccn12,2.3395,36.8145
8,CCOC(=O)CC1(NC(=O)N2Cc3c(sc4c3CCCC4)-n3cccc3C2...,3.4420,3.6015
9,CCc1nc2cc(Br)c(C(=O)OC)nc2n1CC(=O)c1ccccc1,73.5450,1.1345


In [18]:
tmp = pd.merge(processed_duplicate,duplicates,how='left',on='SMILES')
tmp

,SMILES,MLM_x,HLM_x,id,MLM_y,HLM_y,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615,TRAIN_0451,0.310,24.670,3.556,262.309,3,0,4,3.556,43.60
1,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615,TRAIN_2276,1.535,31.453,3.556,262.309,3,0,4,3.556,43.60
2,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110,TRAIN_0543,68.485,85.872,2.172,337.372,4,2,3,2.169,82.00
3,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110,TRAIN_2891,55.950,69.950,2.172,337.372,4,2,3,2.169,82.00
4,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690,TRAIN_0366,73.740,66.850,2.293,367.428,5,2,3,2.307,139.85
5,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690,TRAIN_0837,63.522,62.488,2.293,367.428,5,2,3,2.307,139.85
6,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850,TRAIN_1085,23.630,71.470,1.684,381.450,7,1,4,1.684,126.52
7,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850,TRAIN_2848,40.657,99.900,1.684,381.450,7,1,4,1.684,126.52
8,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385,TRAIN_1666,5.494,25.397,2.843,360.490,3,0,4,2.843,49.85
9,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385,TRAIN_2096,1.880,61.280,2.843,360.490,3,0,4,2.843,49.85


### -> 중복치 제거 (총 26개)

In [19]:
tmp = tmp.iloc[0::2]  # 첫 번째 행부터 시작하여 2씩 증가하여 행 선택
tmp.drop(['MLM_y','HLM_y'],axis =1,inplace=True)

In [20]:
train = train.drop_duplicates(subset=['SMILES'], keep=False) #기존 train에서 중복치들을 우선 모두 제거
train.shape

(3428, 11)

In [21]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,3.409,396.195,3,1,5,3.409,64.74
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,1.912,359.381,4,1,3,1.844,77.37
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,1.941,261.320,3,1,6,2.124,70.14
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.989,284.696,5,1,5,0.989,91.51


In [23]:
tmp

,SMILES,MLM_x,HLM_x,id,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,C(=C/c1nnn(Cc2ccccc2)n1)\c1ccccc1,0.9225,28.0615,TRAIN_0451,3.556,262.309,3,0,4,3.556,43.60
2,CC(=O)Nc1ccc(N2N=C(c3ccc(O)cc3)C(C)CC2=O)cc1,62.2175,77.9110,TRAIN_0543,2.172,337.372,4,2,3,2.169,82.00
4,CC(=O)Nc1nc2ccc(-c3nn(C(C)C)c4nc(N)ncc34)cc2s1,68.6310,64.6690,TRAIN_0366,2.293,367.428,5,2,3,2.307,139.85
6,CC(C)(C)OC(=O)N1CCC(c2n[nH]c3nc(S(C)(=O)=O)ncc...,32.1435,85.6850,TRAIN_1085,1.684,381.450,7,1,4,1.684,126.52
8,CC(C)COc1cc(=O)n2c(c1C(=O)N1CCC(C)CC1)CCCCC2,3.6870,43.3385,TRAIN_1666,2.843,360.490,3,0,4,2.843,49.85
10,CC(C)NC(=O)c1c(Cl)nn(C)c1NC(=O)c1cc(Br)nn1-c1n...,62.1085,68.1015,TRAIN_0550,3.750,501.165,5,2,5,3.750,106.72
12,CC1CC(=O)N(c2ccc(-c3cccc(C#N)c3)cc2)N=C1c1ccc(...,43.1700,31.1300,TRAIN_0328,4.449,381.427,4,1,3,4.446,76.69
14,CCCCC/N=c1\n(C)c(=O)nc2sccn12,2.3395,36.8145,TRAIN_0859,2.484,252.336,4,0,4,2.474,73.56
16,CCOC(=O)CC1(NC(=O)N2Cc3c(sc4c3CCCC4)-n3cccc3C2...,3.4420,3.6015,TRAIN_0565,6.727,537.736,3,1,6,6.727,120.05
18,CCc1nc2cc(Br)c(C(=O)OC)nc2n1CC(=O)c1ccccc1,73.5450,1.1345,TRAIN_0250,3.815,402.242,5,0,6,3.819,74.08


In [24]:
tmp.rename(columns={'HLM_x': 'HLM', 'MLM_x': 'MLM'}, inplace=True)

In [25]:
train = pd.concat([train, tmp], axis=0, ignore_index=True)
train.shape

(3454, 11)

In [26]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.6800,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.5900,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.8920,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.0000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.9900,2.335,268.310,3,0,1,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,24.1955,4.609,340.381,4,1,2,4.736,71.75
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.1260,4.282,553.480,5,0,7,4.009,65.98
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.7320,4.304,459.520,6,1,4,4.304,113.53
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.8400,-1.133,385.417,7,0,4,-1.133,108.65


In [32]:
# train.to_csv(f"{DATA_PATH}train_clean.csv", index=False)

# 데이터 불러오기

## Clean data(예슬)

In [39]:
train_df = pd.read_csv(f'{DATA_PATH}train_clean.csv')
test_df = pd.read_csv(f'{DATA_PATH}test.csv')
train_df

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.6800,3.259,400.495,5,2,8,3.259,117.37
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.5900,2.169,301.407,2,1,2,2.172,73.47
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.8920,1.593,297.358,5,0,3,1.585,62.45
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.0000,4.771,494.652,6,0,5,3.475,92.60
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.9900,2.335,268.310,3,0,1,2.337,42.43
...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,24.1955,4.609,340.381,4,1,2,4.736,71.75
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.1260,4.282,553.480,5,0,7,4.009,65.98
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.7320,4.304,459.520,6,1,4,4.304,113.53
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.8400,-1.133,385.417,7,0,4,-1.133,108.65


In [40]:
train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

(2, 1)

In [41]:
sns.set_style('whitegrid')

# 피쳐 엔지니어링

In [13]:
calc = Calculator(descriptors, ignore_3D=False)
len(calc.descriptors)

1826

In [14]:
x_train = calc.pandas([Chem.MolFromSmiles(x) for x in train_df.SMILES])
x_train

  7%|▋         | 254/3454 [00:31<05:19, 10.02it/s]

c:\python39\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 3454/3454 [03:58<00:00, 14.48it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,35.689316,2.421199,4.745523,35.689316,1.274618,4.249154,...,10.081676,78.761075,400.156912,7.695325,2380,40,142.0,165.0,9.500000,6.361111
1,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,26.575899,2.426398,4.757199,26.575899,1.265519,3.984419,...,9.907828,69.149596,301.124883,7.528122,870,35,112.0,132.0,7.138889,4.527778
2,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,2,1,29.802128,2.510668,4.982923,29.802128,1.354642,4.049690,...,10.144510,70.158066,297.170194,7.248054,1028,36,120.0,145.0,5.277778,4.888889
3,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,45.884166,2.532483,4.973440,45.884166,1.310976,4.500758,...,10.613467,86.199585,494.246395,7.162991,4170,61,192.0,231.0,10.784722,7.500000
4,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,26.308663,2.452930,4.905860,26.308663,1.315433,3.935574,...,9.978363,53.872357,268.121178,7.447810,762,32,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,34.830344,2.505234,4.893210,34.830344,1.339629,4.230453,...,10.344674,77.430819,340.143645,8.098658,1730,43,148.0,180.0,6.611111,5.500000
3450,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,49.727575,2.459083,4.917437,49.727575,1.308620,4.573511,...,10.593756,88.164476,552.169496,8.120140,4874,63,204.0,242.0,12.055556,8.222222
3451,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,42.601740,2.550795,5.037779,42.601740,1.290962,4.451467,...,10.677177,84.226168,459.136511,8.502528,2888,61,186.0,227.0,10.895833,6.944444
3452,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,37.175575,2.442418,4.727992,37.175575,1.327699,4.270653,...,10.177818,79.094663,385.175004,7.552451,2262,42,148.0,175.0,8.000000,6.222222


In [15]:
x_test = calc.pandas([Chem.MolFromSmiles(x) for x in test_df.SMILES])
x_test

100%|██████████| 483/483 [00:24<00:00, 19.89it/s]


,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,32.037339,2.392367,4.778964,32.037339,1.281494,4.137716,...,9.935713,72.422574,361.193631,6.946031,1607,34,126.0,144.0,7.527778,5.638889
1,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,35.824667,2.462328,4.912724,35.824667,1.326840,4.240550,...,10.330421,76.111616,370.152872,7.554140,1853,45,146.0,175.0,7.840278,5.972222
2,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,3,32.761443,2.378577,4.757154,32.761443,1.260056,4.175318,...,10.008163,60.614034,347.174610,7.386694,1905,37,132.0,150.0,8.388889,5.722222
3,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,35.577061,2.496055,4.855755,35.577061,1.368349,4.227377,...,10.288444,77.464779,345.133808,8.417898,1673,42,146.0,177.0,6.000000,5.583333
4,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,34.235212,2.546495,4.990622,34.235212,1.316739,4.210465,...,10.380312,75.874629,353.185175,7.207861,1656,47,144.0,176.0,8.722222,5.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,28.248662,2.381151,4.715751,28.248662,1.284030,3.995519,...,9.862509,69.656439,306.230728,5.889052,1138,32,108.0,125.0,7.756944,5.222222
479,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,1,29.983837,2.559820,5.003321,29.983837,1.249327,4.116090,...,10.288852,73.647870,335.184506,6.840500,1280,42,130.0,157.0,9.201389,5.180556
480,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,33.716988,2.612365,5.113784,33.716988,1.296807,4.210713,...,10.447235,76.277596,349.142641,7.758725,1469,48,144.0,177.0,8.722222,5.611111
481,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,0,0,24.141151,2.379239,4.758478,24.141151,1.207058,3.909645,...,9.874728,53.565348,339.985883,11.332863,826,31,102.0,118.0,8.388889,4.361111


In [42]:
# 복사
train_x = x_train.copy()
test_x = x_test.copy()

# invalid value encountered in scalar divide 로 채워진 행 삭제
train_x = train_x.drop(1396).reset_index(drop=True)

# obj_cols
obj_cols = train_x.select_dtypes(include=['object']).columns
obj_cols = obj_cols.union(test_x.select_dtypes(include=['object']).columns) # 테스트셋과의 합집합
obj_cols

Index(['AATS6Z', 'AATS6are', 'AATS6d', 'AATS6dv', 'AATS6i', 'AATS6m', 'AATS6p',
       'AATS6pe', 'AATS6s', 'AATS6se',
       ...
       'WNSA1', 'WNSA2', 'WNSA3', 'WNSA4', 'WNSA5', 'WPSA1', 'WPSA2', 'WPSA3',
       'WPSA4', 'WPSA5'],
      dtype='object', length=577)

In [43]:
train_x[obj_cols] = train_x[obj_cols].astype(float)
test_x[obj_cols] = test_x[obj_cols].astype(float)

# train_x = train_x.drop(columns=obj_cols)
# test_x = test_x.drop(columns=obj_cols)

In [107]:
# # 엑셀로도 저장해서 확인해보기
# with pd.ExcelWriter(f"{DATA_PATH}obj.xlsx", engine='openpyxl') as writer:
#     train_x[obj_cols].to_excel(writer, sheet_name='시트1', index=False)

In [44]:
# 결측치 채우기
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [45]:
# 0 으로만 구성된 컬럼은 버리기
train_x = train_x.loc[:, (train_x != 0).any(axis=0)]
train_x

# 테스트셋에 적용
cols = train_x.loc[:, (train_x != 0).any(axis=0)].columns
test_x = test_x[cols]

In [46]:
bool_cols = train_x.select_dtypes(include=['bool']).columns
bool_cols

# bool type 숫자로 변환
train_x[bool_cols] = train_x[bool_cols].astype(int)
test_x[bool_cols] = test_x[bool_cols].astype(int)

# drop
# train_x = train_x.drop(columns=bool_cols)
# test_x = test_x.drop(columns=bool_cols)

In [ ]:
# float_cols = train_x.select_dtypes(include=['float64']).columns

# train_x[float_cols] = train_x[float_cols].astype(int)
# test_x[float_cols] = test_x[float_cols].astype(int)
# train_x.info()

## 알파벳 기준으로 평균화

In [ ]:
# train_group = train_x.groupby(train_x.columns.str[0], axis=1)
# train_mean = train_group.mean()

# test_group = test_x.groupby(test_x.columns.str[0], axis=1)
# test_mean = test_group.mean()

# train_mean

## 분위수로 범주화

In [ ]:
# # 테스트셋
# for col in train_mean.columns:
#     quantile = train_mean[col].quantile([0,0.2,0.4,0.6,0.8,1])
#     label = [1,2,3,4,5]
#     train_mean[col] = pd.cut(train_mean[col], bins=quantile.values, labels=label, include_lowest=True)

# train_mean = train_mean.astype(int)
# train_mean

In [ ]:
# # 테스트셋
# for col in test_mean.columns:
#     quantile = test_mean[col].quantile([0,0.2,0.4,0.6,0.8,1])
#     label = [1,2,3,4,5]
#     test_mean[col] = pd.cut(test_mean[col], bins=quantile.values, labels=label, include_lowest=True)

# test_mean = test_mean.astype(int)
# test_mean

In [59]:
# len(train_mean.columns) , len(test_mean.columns)

(28, 28)

## 합치기

In [47]:
train_df = pd.concat([train_df, train_x], axis=1)
test_df = pd.concat([test_df, test_x], axis=1)
train_df

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,50.6800,3.259,400.495,5,2,8,3.259,...,10.081676,78.761075,400.156912,7.695325,2380.0,40.0,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,50.5900,2.169,301.407,2,1,2,2.172,...,9.907828,69.149596,301.124883,7.528122,870.0,35.0,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,80.8920,1.593,297.358,5,0,3,1.585,...,10.144510,70.158066,297.170194,7.248054,1028.0,36.0,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,2.0000,4.771,494.652,6,0,5,3.475,...,10.613467,86.199585,494.246395,7.162991,4170.0,61.0,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,99.9900,2.335,268.310,3,0,1,2.337,...,9.978363,53.872357,268.121178,7.447810,762.0,32.0,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,24.1955,4.609,340.381,4,1,2,4.736,...,10.593756,88.164476,552.169496,8.120140,4874.0,63.0,204.0,242.0,12.055556,8.222222
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,66.1260,4.282,553.480,5,0,7,4.009,...,10.677177,84.226168,459.136511,8.502528,2888.0,61.0,186.0,227.0,10.895833,6.944444
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,63.7320,4.304,459.520,6,1,4,4.304,...,10.177818,79.094663,385.175004,7.552451,2262.0,42.0,148.0,175.0,8.000000,6.222222
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,93.8400,-1.133,385.417,7,0,4,-1.133,...,10.410937,83.783308,466.102099,8.631520,3644.0,49.0,170.0,200.0,9.791667,6.916667


In [ ]:
# train_df = pd.concat([train_df, train_x[bool_cols], train_mean], axis=1)
# test_df = pd.concat([test_df, test_x[bool_cols], test_mean], axis=1)
# train_df

In [48]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3454 entries, 0 to 3453
Columns: 1383 entries, id to mZagreb2
dtypes: float64(1378), int64(3), object(2)
memory usage: 36.4+ MB


In [49]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

(0, 0)

# 정답 제거

In [50]:
# train_df = train_df.drop(columns='MLM')
train_df = train_df.drop(columns='HLM')
train_df

,id,SMILES,MLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.0100,3.259,400.495,5,2,8,3.259,117.37,...,10.081676,78.761075,400.156912,7.695325,2380.0,40.0,142.0,165.0,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.2700,2.169,301.407,2,1,2,2.172,73.47,...,9.907828,69.149596,301.124883,7.528122,870.0,35.0,112.0,132.0,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.5860,1.593,297.358,5,0,3,1.585,62.45,...,10.144510,70.158066,297.170194,7.248054,1028.0,36.0,120.0,145.0,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.7100,4.771,494.652,6,0,5,3.475,92.60,...,10.613467,86.199585,494.246395,7.162991,4170.0,61.0,192.0,231.0,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.2700,2.335,268.310,3,0,1,2.337,42.43,...,9.978363,53.872357,268.121178,7.447810,762.0,32.0,106.0,125.0,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3449,TRAIN_1833,Cc1ccc(-c2cc(-c3nc4c5ccccc5ncn4n3)[nH]n2)c(C)c1,28.4630,4.609,340.381,4,1,2,4.736,71.75,...,10.593756,88.164476,552.169496,8.120140,4874.0,63.0,204.0,242.0,12.055556,8.222222
3450,TRAIN_0898,Cc1ccc(-c2ccc(C(CN3CCCC3)N(C)C(=O)CN3C(=O)COc4...,56.0435,4.282,553.480,5,0,7,4.009,65.98,...,10.677177,84.226168,459.136511,8.502528,2888.0,61.0,186.0,227.0,10.895833,6.944444
3451,TRAIN_2575,Cc1ccccc1-c1nc2ccccc2cc1C(C)n1c(=O)[nH]c2c(S(C...,56.3130,4.304,459.520,6,1,4,4.304,113.53,...,10.177818,79.094663,385.175004,7.552451,2262.0,42.0,148.0,175.0,8.000000,6.222222
3452,TRAIN_0216,N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1,70.2150,-1.133,385.417,7,0,4,-1.133,108.65,...,10.410937,83.783308,466.102099,8.631520,3644.0,49.0,170.0,200.0,9.791667,6.916667


In [51]:
test_df

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,2.641,361.505,4,2,7,2.635,92.76,0,...,9.935713,72.422574,361.193631,6.946031,1607,34,126.0,144.0,7.527778,5.638889
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.585,370.399,5,0,3,0.585,68.31,0,...,10.330421,76.111616,370.152872,7.554140,1853,45,146.0,175.0,7.840278,5.972222
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,4.276,347.414,4,4,5,4.290,92.86,0,...,10.008163,60.614034,347.174610,7.386694,1905,37,132.0,150.0,8.388889,5.722222
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,1.795,345.358,5,0,2,1.795,81.21,0,...,10.288444,77.464779,345.133808,8.417898,1673,42,146.0,177.0,6.000000,5.583333
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,1.219,353.418,4,0,2,0.169,61.15,0,...,10.380312,75.874629,353.185175,7.207861,1656,47,144.0,176.0,8.722222,5.638889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,4.207,306.443,2,1,7,4.207,55.13,0,...,9.862509,69.656439,306.230728,5.889052,1138,32,108.0,125.0,7.756944,5.222222
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,-0.608,335.398,5,0,1,-1.736,70.16,0,...,10.288852,73.647870,335.184506,6.840500,1280,42,130.0,157.0,9.201389,5.180556
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,1.792,349.383,3,1,3,1.792,69.72,0,...,10.447235,76.277596,349.142641,7.758725,1469,48,144.0,177.0,8.722222,5.611111
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0.790,341.132,3,2,2,0.423,69.64,0,...,9.874728,53.565348,339.985883,11.332863,826,31,102.0,118.0,8.388889,4.361111


In [26]:
!pip install imblearn

# 오버샘플링

In [52]:
from imblearn.combine import SMOTEENN

def smoteenn_combine(df):
    X = df.iloc[:,3:]
    y = df['MLM'].astype(int) # 연속형 -> 이산형
    num_resampled = round(train_df.shape[0] / 2) # 절반
    smoteenn = SMOTEENN(sampling_strategy={1:num_resampled}, random_state=SEED)
    # smoteenn = SMOTEENN(sampling_strategy='auto', random_state=SEED)
    X_resampled, y_resampled = smoteenn.fit_resample(X, y)

    X_df = pd.DataFrame(X_resampled, columns=X.columns)
    y_df = pd.DataFrame({'MLM': y_resampled})

    return X_df, y_df

X_resampled, y_resampled = smoteenn_combine(train_df)

In [53]:
train_over = pd.concat([y_resampled,X_resampled],axis=1)
train_df = pd.concat([train_df,train_over],axis=0).reset_index(drop=True)
train_df

,id,SMILES,MLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,3.259000,400.495000,5,2,8,3.259000,117.370000,...,10.081676,78.761075,400.156912,7.695325,2380.000000,40.000000,142.000000,165.000000,9.500000,6.361111
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,2.169000,301.407000,2,1,2,2.172000,73.470000,...,9.907828,69.149596,301.124883,7.528122,870.000000,35.000000,112.000000,132.000000,7.138889,4.527778
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,1.593000,297.358000,5,0,3,1.585000,62.450000,...,10.144510,70.158066,297.170194,7.248054,1028.000000,36.000000,120.000000,145.000000,5.277778,4.888889
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,4.771000,494.652000,6,0,5,3.475000,92.600000,...,10.613467,86.199585,494.246395,7.162991,4170.000000,61.000000,192.000000,231.000000,10.784722,7.500000
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,2.335000,268.310000,3,0,1,2.337000,42.430000,...,9.978363,53.872357,268.121178,7.447810,762.000000,32.000000,106.000000,125.000000,6.277778,4.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,NaN,NaN,1.000,1.394193,409.217060,5,0,6,1.394193,107.858701,...,10.434699,82.580395,482.077495,8.457500,3486.626533,50.985073,170.626862,199.597008,12.073798,6.933250
5112,NaN,NaN,1.000,2.976032,423.024921,3,1,5,2.708228,105.297582,...,10.479039,71.381617,414.792244,8.159345,2485.931425,50.000000,161.493253,193.000000,8.774161,6.167527
5113,NaN,NaN,1.000,4.115837,382.087736,2,0,3,4.115837,44.202938,...,10.380000,78.418709,458.564245,7.163908,3815.124029,58.019582,176.039164,206.097911,10.761047,7.577758
5114,NaN,NaN,1.000,4.106384,349.740332,3,1,5,4.108809,61.182108,...,9.985149,68.869707,276.898072,9.985291,750.683238,30.000000,108.478318,131.043365,5.006095,4.181778


# 언더샘플링

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler

# def random_undersample(df):
#     X = df.iloc[:,3:]
#     y = df['HLM'].astype(int) # 연속형 -> 이산형
    
#     # sampler = RandomUnderSampler(sampling_strategy='auto', random_state=SEED)
#     sampler = RandomUnderSampler(sampling_strategy='majority', random_state=SEED) # 많은 클래스의 샘플은 모두 유지, 적은 클래스의 샘플은 일부만 유지
#     X_resampled, y_resampled = sampler.fit_resample(X, y)

#     X_df = pd.DataFrame(X_resampled, columns=X.columns)
#     y_df = pd.DataFrame({'HLM': y_resampled})

#     return X_resampled, y_resampled

# # 사용 예시
# X_resampled, y_resampled = random_undersample(train_df)
# X_resampled.shape

In [ ]:
# train_under = pd.concat([y_resampled,X_resampled],axis=1)
# train_under

In [ ]:
# # 정답값 실제데이터로 반영(이산형으로 바꾸려고 정수로 변환했어가지고)
# train_under['HLM'] = train_df['HLM']
# train_under

In [ ]:
# train_under.isnull().sum().sum()

# Scaling

In [54]:
train_df.iloc[:,3:].columns

Index(['AlogP', 'Molecular_Weight', 'Num_H_Acceptors', 'Num_H_Donors',
       'Num_RotatableBonds', 'LogD', 'Molecular_PolarSurfaceArea', 'nAcid',
       'nBase', 'SpAbs_A',
       ...
       'SRW10', 'TSRW10', 'MW', 'AMW', 'WPath', 'WPol', 'Zagreb1', 'Zagreb2',
       'mZagreb1', 'mZagreb2'],
      dtype='object', length=1379)

In [55]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_cols = train_df.iloc[:,3:].columns
# train_cols = train_under.iloc[:,1:].columns # 언더샘플링된 학습데이터
test_cols = test_df.iloc[:,2:].columns

train_df[train_cols] = scaler.fit_transform(train_df[train_cols])
# train_under[train_cols] = scaler.fit_transform(train_under[train_cols]) # 언더샘플링된 학습데이터
test_df[test_cols] = scaler.transform(test_df[test_cols])

test_df

,id,SMILES,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,nAcid,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,0.435626,0.338954,0.333333,0.250,0.411765,0.476103,0.428921,0.0,...,0.869813,0.676470,0.494571,0.317098,0.150440,0.320755,0.420000,0.384000,0.339812,0.506866
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0.258445,0.354877,0.416667,0.000,0.176471,0.312260,0.311772,0.0,...,0.904367,0.710928,0.506839,0.344860,0.173469,0.424528,0.486667,0.466667,0.353918,0.536829
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,0.576525,0.313726,0.333333,0.500,0.294118,0.608376,0.429400,0.0,...,0.876156,0.566171,0.475376,0.337215,0.178337,0.349057,0.440000,0.400000,0.378683,0.514357
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,0.362720,0.310045,0.416667,0.000,0.117647,0.408967,0.373581,0.0,...,0.900693,0.723568,0.472581,0.384292,0.156619,0.396226,0.486667,0.472000,0.270846,0.501873
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,0.313082,0.324475,0.333333,0.000,0.117647,0.279012,0.277466,0.0,...,0.908735,0.708715,0.483606,0.329051,0.155027,0.443396,0.480000,0.469333,0.393730,0.506866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,0.570579,0.240375,0.166667,0.125,0.411765,0.601742,0.248622,0.0,...,0.863404,0.650633,0.419312,0.268845,0.106534,0.301887,0.360000,0.333333,0.350157,0.469413
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,0.155636,0.292214,0.416667,0.000,0.058824,0.126758,0.320636,0.0,...,0.900728,0.687915,0.458958,0.312281,0.119828,0.396226,0.433333,0.418667,0.415361,0.465668
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,0.362461,0.317251,0.250000,0.125,0.176471,0.408728,0.318528,0.0,...,0.914594,0.712479,0.478070,0.354199,0.137521,0.452830,0.480000,0.472000,0.393730,0.504370
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0.276112,0.302479,0.250000,0.250,0.117647,0.299313,0.318145,0.0,...,0.864474,0.500333,0.465532,0.517365,0.077326,0.292453,0.340000,0.314667,0.378683,0.392010


In [56]:
train_df

,id,SMILES,MLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,0.488883,0.408759,0.416667,0.250,0.470588,0.525975,0.546835,...,0.882591,0.735676,0.547923,0.351305,0.222805,0.377358,0.473333,0.440000,0.428840,0.571785
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,0.394950,0.231359,0.166667,0.125,0.117647,0.439098,0.336496,...,0.867372,0.645899,0.412321,0.343672,0.081445,0.330189,0.373333,0.352000,0.322257,0.406991
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,0.345312,0.224110,0.416667,0.000,0.176471,0.392184,0.283695,...,0.888092,0.655318,0.406906,0.330886,0.096237,0.339623,0.400000,0.386667,0.238245,0.439451
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,0.619183,0.577331,0.500000,0.000,0.294118,0.543238,0.428154,...,0.929146,0.805156,0.676756,0.327003,0.390376,0.575472,0.640000,0.616000,0.486834,0.674157
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,0.409255,0.172104,0.250000,0.000,0.058824,0.452286,0.187773,...,0.873547,0.503200,0.367130,0.340005,0.071335,0.301887,0.353333,0.333333,0.283386,0.392010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,NaN,NaN,1.000,0.328179,0.424374,0.416667,0.000,0.352941,0.376934,0.501263,...,0.913496,0.771350,0.660094,0.386099,0.326402,0.480991,0.568756,0.532259,0.545024,0.623214
5112,NaN,NaN,1.000,0.464498,0.449095,0.250000,0.125,0.294118,0.481956,0.488992,...,0.917378,0.666747,0.567962,0.372488,0.232722,0.471698,0.538311,0.514667,0.396075,0.554384
5113,NaN,NaN,1.000,0.562723,0.375804,0.166667,0.000,0.176471,0.594456,0.196267,...,0.908708,0.732478,0.627898,0.327045,0.357154,0.547355,0.586797,0.549594,0.485765,0.681147
5114,NaN,NaN,1.000,0.561908,0.317891,0.250000,0.125,0.294118,0.593895,0.277620,...,0.874141,0.643284,0.379148,0.455846,0.070276,0.283019,0.361594,0.349449,0.225980,0.375890


In [57]:
train_df.to_csv(f"{DATA_PATH}train_clean_mor_scale_12.csv", index=False)
# train_under.to_csv(f"{DATA_PATH}train_clean_mor_scale_8.csv", index=False)
test_df.to_csv(f"{DATA_PATH}test_clean_mor_scale_12.csv", index=False)

In [10]:
train_df = pd.read_csv(f'{DATA_PATH}train_clean_mor_scale_11.csv')
test_df = pd.read_csv(f'{DATA_PATH}test_clean_mor_scale_11.csv')
train_df

,id,SMILES,MLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,0.488883,0.408759,0.416667,0.250,0.470588,0.525975,0.546835,...,0.882591,0.735676,0.547923,0.351305,0.222805,0.377358,0.473333,0.440000,0.428840,0.571785
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,0.394950,0.231359,0.166667,0.125,0.117647,0.439098,0.336496,...,0.867372,0.645899,0.412321,0.343672,0.081445,0.330189,0.373333,0.352000,0.322257,0.406991
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,0.345312,0.224110,0.416667,0.000,0.176471,0.392184,0.283695,...,0.888092,0.655318,0.406906,0.330886,0.096237,0.339623,0.400000,0.386667,0.238245,0.439451
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,0.619183,0.577331,0.500000,0.000,0.294118,0.543238,0.428154,...,0.929146,0.805156,0.676756,0.327003,0.390376,0.575472,0.640000,0.616000,0.486834,0.674157
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,0.409255,0.172104,0.250000,0.000,0.058824,0.452286,0.187773,...,0.873547,0.503200,0.367130,0.340005,0.071335,0.301887,0.353333,0.333333,0.283386,0.392010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51484,NaN,NaN,99.000,0.328174,0.226472,0.416667,0.125,0.176471,0.376382,0.457413,...,0.869330,0.671487,0.435614,0.382483,0.145414,0.301887,0.419820,0.386559,0.233077,0.474138
51485,NaN,NaN,99.000,0.333077,0.381763,0.333333,0.125,0.117647,0.400663,0.452254,...,0.908217,0.675696,0.544631,0.407531,0.193439,0.451107,0.508077,0.483385,0.454886,0.549133
51486,NaN,NaN,99.000,0.374032,0.329143,0.416667,0.125,0.176471,0.307571,0.522571,...,0.871317,0.661961,0.420419,0.389357,0.127036,0.301887,0.406667,0.378667,0.221944,0.454432
51487,NaN,NaN,99.000,0.323107,0.288004,0.416667,0.125,0.176471,0.329566,0.493303,...,0.888884,0.687831,0.456970,0.361466,0.102535,0.347541,0.412262,0.396048,0.375834,0.447498


# Auto Modeling

In [12]:
from flaml import AutoML
model = AutoML()
params = {
    "metric": "rmse",
    "task": "regression",
    "time_budget": 60*60*4,
    "seed": SEED,
    "early_stop": False,
    "ensemble" : True,
    # "estimator_list": ['lgbm']
    "estimator_list": ['lgbm', 'xgboost', 'rf', 'catboost']
}

model.fit(train_df.iloc[:,3:], train_df['MLM'], **params)
# model.fit(train_df.iloc[:,1:], train_df['HLM'], **params)
pred = model.predict(test_df.iloc[:,2:])
print(model.best_config)
print(model.best_loss)

[flaml.automl.logger: 09-24 11:54:44] {1679} INFO - task = regression
[flaml.automl.logger: 09-24 11:54:44] {1690} INFO - Evaluation method: holdout
[flaml.automl.logger: 09-24 11:54:46] {1788} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 09-24 11:54:46] {1900} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'rf', 'catboost']
[flaml.automl.logger: 09-24 11:54:46] {2218} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 09-24 11:54:49] {2344} INFO - Estimated sufficient time budget=116572s. Estimated necessary time budget=228s.
[flaml.automl.logger: 09-24 11:54:49] {2391} INFO -  at 24.3s,	estimator lgbm's best error=27.8509,	best estimator lgbm's best error=27.8509
[flaml.automl.logger: 09-24 11:54:49] {2218} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 09-24 11:54:52] {2391} INFO -  at 27.3s,	estimator lgbm's best error=27.8509,	best estimator lgbm's best error=27.8509
[flaml.automl.logger: 09-24 11:54:52] {2218} INFO - iterat

- 1 : 32.53133291308737 : 알파벳 기준 평균화
- 2 : 32.50343224854824 : 평균->범주화, bool 타입 살리기
- 3 : 32.598785545609914 : 알파벳 앞 2개 기준 평균화, bool 타입 살리기
- 4 : 32.59231590200649 : 알파벳 앞 2개 기준 평균화, bool 타입 살리기, int
- 5 : 13.68499210379137 : 평균->범주화, bool 타입 살리기, Smoteenn(32142개)
- 6 : 27.66777921166814 : 평균->범주화, bool 타입 살리기, Smoteenn(1500개)
- 7 : 27점대 : 알파벳x(0만 drop), bool 타입 살리기, 언더샘플링(최종1100개)
- 8 : 30.371972268239386 : 알파벳x(0만 drop), bool 타입 살리기, 언더샘플링(최종3000개)
- 여기서부터 MLM 예측
- 9 : 12.62284755465401 : 평균->범주화, bool 타입 살리기, Smoteenn(40000개)
- 10 : 27.088651919701498 : 평균->범주화, bool 타입 살리기, Smoteenn(1500개)
- 11 : 12.400419435221954 : 알파벳x(0만 drop), bool 타입 살리기, Smoteenn(50000개)
- 12 : 27.482319519422198 : 알파벳x(0만 drop), bool 타입 살리기, Smoteenn(1500개)
- 13 : 8.055969577263326 : 11번 df의 AutoML 앙상블
- 14 : 26.365826007126714 : 12번 df의 AutoML 앙상블

In [18]:
pred

array([ 32.72863215,  37.93483811,  46.55202314,  62.0941466 ,
        51.66770505,  30.69454544,  88.84810481,  60.89144793,
        47.18374348,  54.91305917,  48.12402119,  29.93919415,
        46.95823075,  63.09190124,  23.43436768,  55.39544476,
        -0.46841799,  38.32677814,  53.32674748,  52.69338214,
        36.31397436,  42.9912844 ,   6.18321104,  33.08343241,
        46.30036651,  42.04782669,  43.91961091,  56.0755384 ,
        51.38312419,  91.06380045,  42.06081591,  25.77305003,
        27.50545905,  34.99580375,  43.70146032,  38.00907728,
        28.86432773,  39.5209068 ,  69.12445678,  47.70310966,
        29.86483982,  57.38774911,  34.6866553 ,  37.01468953,
        23.28840293,  45.31838237,  36.80692129,  31.07135235,
        67.41699207,  38.58148312,  62.74920936,  36.04565015,
        53.53564236,  51.15524116,  25.54147473,  62.53037252,
        70.66674274,  34.88981456,  39.97946082,  42.56571282,
        27.71134075,  55.48320118,  13.56868531,  62.38

# 제출

In [14]:
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submission

,id,MLM,HLM
0,TEST_000,0,0
1,TEST_001,0,0
2,TEST_002,0,0
3,TEST_003,0,0
4,TEST_004,0,0
...,...,...,...
478,TEST_478,0,0
479,TEST_479,0,0
480,TEST_480,0,0
481,TEST_481,0,0


In [15]:
# submission['HLM'] = pred
submission['MLM'] = pred
submission

,id,MLM,HLM
0,TEST_000,32.728632,0
1,TEST_001,37.934838,0
2,TEST_002,46.552023,0
3,TEST_003,62.094147,0
4,TEST_004,51.667705,0
...,...,...,...
478,TEST_478,35.040958,0
479,TEST_479,73.527388,0
480,TEST_480,42.757677,0
481,TEST_481,60.555713,0


In [16]:
submission.to_csv(f"{DATA_PATH}aml_13_ensemble.csv", index=False)
submission.to_csv("aml_13_ensemble.csv", index=False)

In [ ]:
ensemble = pd.read_csv(f"{DATA_PATH}ensemble.csv")
comparison = pd.concat([ensemble['HLM'],submission['HLM']], axis=1)

# 엑셀로 확인
with pd.ExcelWriter("comparison.xlsx", engine='openpyxl') as writer:
    comparison.to_excel(writer, sheet_name='시트1', index=False)